### como temos poucos dados testes, por termos poucos dados no geral, os accuracy oscilam muito

### modelos


- ~~Logistic Regression (non-regularized)~~

- ~~Logistic Regression (regularized)~~

    - ~~using ridge~~

    - ~~using lasso~~


- ~~Redes Neuronais (não regularizada)~~

- ~~Redes Neuronais (regularizada)~~

- ~~Support Vector Classification (SVC) (nao regularizado)~~

- ~~Support Vector Classification (SVC) (regularizado)~~

In [272]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn import datasets
from sklearn.svm import SVC

In [273]:
stats = {}

data_file = '02balanced.csv'
#data_file = '04onlyFatig.csv'


stats['data_file'] = data_file
data = pd.read_csv(data_file)
data.sample(5)

,Name,Date,Sex_F,Appetite,SleepInd,StSpirit,pEffort,uaI,uaI(-1),DeltaWt (%),DeltaHrtwo,FatigInd
316,I_f,2019-12-31,1,6.0,-5.0,4.0,1.0,0.00,1.41,0.000000,0.0,Risk/Caution
65,J_f,2020-04-08,1,6.0,-5.5,5.0,2.0,2.00,2.00,-0.002227,6.0,Low/Minimal
118,F_m,2019-11-08,0,4.0,-3.0,4.0,3.0,1.69,2.31,0.000000,10.0,Optimal
4,F_m,2020-05-10,0,3.0,-3.5,5.0,1.0,0.00,2.00,0.006838,14.0,Low/Minimal
75,J_f,2020-04-03,1,6.0,-6.0,5.0,2.0,2.00,2.00,0.002227,0.0,Low/Minimal


In [274]:
def standardize_numeric_columns(df):
    # Separate numeric and non-numeric columns
    numeric_cols = ['Appetite', 'SleepInd', 'StSpirit', 'pEffort',
                    'uaI', 'uaI(-1)', 'DeltaWt (%)', 'DeltaHrtwo']

    # Initialize StandardScaler
    scaler = StandardScaler()

    # Standardize only the numeric columns
    df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

    # Display the resulting DataFrame
    return df 

df = standardize_numeric_columns(data)

In [275]:
def logistical_regression(df, sex="all", only_accuracy=False):
       """regressao logistica apresentada não está regularizada"""
       if sex=="all":
              filtered_df = df
       elif sex=="F":
              filtered_df = df[df["Sex_F"] > 0.5]
       elif sex=="M":
              filtered_df = df[df["Sex_F"] < 0.5]
       else:
              raise ValueError("Invalid")


       X = filtered_df[['Sex_F', 'Appetite', 'SleepInd', 'StSpirit', 'pEffort',
              'uaI', 'uaI(-1)', 'DeltaWt (%)', 'DeltaHrtwo']]
       y = filtered_df['FatigInd']

       X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
       # Initialize the model
       model = LogisticRegression()

       # Train the model
       model.fit(X_train, y_train)

       # Make predictions
       y_pred = model.predict(X_test)
       print("Accuracy")
       print(accuracy_score(y_test, y_pred))


       # Display confusion matrix
       print("Confusion Matrix:")
       print(confusion_matrix(y_test, y_pred))

       """
       metrics = {
              "Logistic Regression": {
                     "Accuracy": accuracy_score(y_test, y_pred),
                     "F1 Score": f1_score(y_test, y_pred, average='weighted'),
                     "Precision": precision_score(y_test, y_pred, average='weighted'),
                     "Recall": recall_score(y_test, y_pred, average='weighted')
              }}
       print(metrics)
       """

       if not only_accuracy:
              # Classification report for precision, recall, and F1-score
              print("Classification Report:")
              print(classification_report(y_test, y_pred))
              # Display coefficients of the model (for each feature)
              print("Model Coefficients:", model.coef_)
              print("Intercept:", model.intercept_)

       return accuracy_score(y_test, y_pred)

a = logistical_regression(df, sex="all", only_accuracy=True)
b = logistical_regression(df, sex="F", only_accuracy=True)
c = logistical_regression(df, sex="M", only_accuracy=True)

stats['logistic_regression_all'] = a
stats['logistic_regression_F'] = b
stats['logistic_regression_M'] = c

Accuracy
0.5925925925925926
Confusion Matrix:
[[ 8  1  0]
 [ 6  8 10]
 [ 0  5 16]]
Accuracy
0.5185185185185185
Confusion Matrix:
[[5 4 0]
 [2 3 1]
 [1 5 6]]
Accuracy
0.7037037037037037
Confusion Matrix:
[[9 1 1]
 [3 5 0]
 [0 3 5]]


In [276]:
def logistical_regression_comPenalidade_RIDGE(df, sex="all", only_accuracy=False):
       """regressao logistica apresentada está regularizada:
       using Ridge (L2) regularization
       C parameter controls the strength of the regularization (with smaller values indicating stronger regularization
       # C is the inverse of regularization strength"""
       if sex=="all":
              filtered_df = df
       elif sex=="F":
              filtered_df = df[df["Sex_F"] > 0.5]
       elif sex=="M":
              filtered_df = df[df["Sex_F"] < 0.5]
       else:
              raise ValueError("Invalid")


       X = filtered_df[['Sex_F', 'Appetite', 'SleepInd', 'StSpirit', 'pEffort',
              'uaI', 'uaI(-1)', 'DeltaWt (%)', 'DeltaHrtwo']]
       y = filtered_df['FatigInd']

       X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
       # Initialize the model
       model = LogisticRegression(penalty='l2', C=0.1)

       # Train the model
       model.fit(X_train, y_train)

       # Make predictions
       y_pred = model.predict(X_test)

       # Calculate accuracy
       accuracy = accuracy_score(y_test, y_pred)
       print(f'Accuracy: {accuracy}')
       #accuracy = accuracy_score(y_train, model.predict(X_train))
       #print(f'Accuracy: {accuracy}')

       # Display confusion matrix
       print("Confusion Matrix:")
       print(confusion_matrix(y_test, y_pred))

       if not only_accuracy:
              # Classification report for precision, recall, and F1-score
              print("Classification Report:")
              print(classification_report(y_test, y_pred))
              # Display coefficients of the model (for each feature)
              print("Model Coefficients:", model.coef_)
              print("Intercept:", model.intercept_)

       return accuracy

a = logistical_regression_comPenalidade_RIDGE(df, sex="all", only_accuracy=True)
b = logistical_regression_comPenalidade_RIDGE(df, sex="F", only_accuracy=True)
c = logistical_regression_comPenalidade_RIDGE(df, sex="M", only_accuracy=True)

stats['logistic_regression_comPenalidade_RIDGE_all'] = a
stats['logistic_regression_comPenalidade_RIDGE_F'] = b
stats['logistic_regression_comPenalidade_RIDGE_M'] = c

Accuracy: 0.6481481481481481
Confusion Matrix:
[[13  1  3]
 [ 4  7  6]
 [ 2  3 15]]
Accuracy: 0.7037037037037037
Confusion Matrix:
[[8 0 2]
 [3 4 2]
 [0 1 7]]
Accuracy: 0.7407407407407407
Confusion Matrix:
[[7 0 1]
 [1 6 4]
 [0 1 7]]


In [277]:
def logistical_regression_comPenalidade_LASSO(df, sex="all", only_accuracy=False):
       """regressao logistica apresentada está regularizada:
       using Lasso (L1) regularization
       C parameter controls the strength of the regularization (with smaller values indicating stronger regularization
       # C is the inverse of regularization strength"""
       if sex=="all":
              filtered_df = df
       elif sex=="F":
              filtered_df = df[df["Sex_F"] > 0.5]
       elif sex=="M":
              filtered_df = df[df["Sex_F"] < 0.5]
       else:
              raise ValueError("Invalid")


       X = filtered_df[['Sex_F', 'Appetite', 'SleepInd', 'StSpirit', 'pEffort',
              'uaI', 'uaI(-1)', 'DeltaWt (%)', 'DeltaHrtwo']]
       y = filtered_df['FatigInd']

       X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
       # Initialize the model
       model = LogisticRegression(solver='liblinear', penalty='l1')

       # Train the model
       model.fit(X_train, y_train)

       # Make predictions
       y_pred = model.predict(X_test)

       # Calculate accuracy
       accuracy = accuracy_score(y_test, y_pred)
       print(f'Accuracy: {accuracy}')
       #accuracy = accuracy_score(y_train, model.predict(X_train))
       #print(f'Accuracy: {accuracy}')

       # Display confusion matrix
       print("Confusion Matrix:")
       print(confusion_matrix(y_test, y_pred))

       if not only_accuracy:
              # Classification report for precision, recall, and F1-score
              print("Classification Report:")
              print(classification_report(y_test, y_pred))
              # Display coefficients of the model (for each feature)
              print("Model Coefficients:", model.coef_)
              print("Intercept:", model.intercept_)
       
       return accuracy

a = logistical_regression_comPenalidade_LASSO(df, sex="all", only_accuracy=True)
b = logistical_regression_comPenalidade_LASSO(df, sex="F", only_accuracy=True)
c = logistical_regression_comPenalidade_LASSO(df, sex="M", only_accuracy=True)

stats['logistic_regression_comPenalidade_LASSO_all'] = a
stats['logistic_regression_comPenalidade_LASSO_F'] = b
stats['logistic_regression_comPenalidade_LASSO_M'] = c

Accuracy: 0.5
Confusion Matrix:
[[11  3  1]
 [ 6  8  8]
 [ 2  7  8]]
Accuracy: 0.37037037037037035
Confusion Matrix:
[[7 0 4]
 [4 0 5]
 [1 3 3]]
Accuracy: 0.5925925925925926
Confusion Matrix:
[[9 1 1]
 [7 3 1]
 [0 1 4]]


In [278]:
def neural_network(df, sex="all"):
        """rede neural apresentada não está regularizada"""
        if sex=="all":
                filtered_df = df
        elif sex=="F":
                filtered_df = df[df["Sex_F"] > 0.5]
        elif sex=="M":
                filtered_df = df[df["Sex_F"] < 0.5]
        else:
                raise ValueError("Invalid")

        X = filtered_df[['Sex_F', 'Appetite', 'SleepInd', 'StSpirit', 'pEffort',
                'uaI', 'uaI(-1)', 'DeltaWt (%)', 'DeltaHrtwo']]
        y = filtered_df['FatigInd']

        # Split data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


        # Neural Network Model
        nn = MLPClassifier(hidden_layer_sizes=(64, 64), max_iter=1000)
        nn.fit(X_train, y_train)
        nn_pred = nn.predict(X_test)
        nn_probs = nn.predict_proba(X_test)  # Probabilities for ROC-AUC

        metrics = {
                "Accuracy": accuracy_score(y_test, nn_pred),
                "F1 Score": f1_score(y_test, nn_pred, average='weighted'),
                "Precision": precision_score(y_test, nn_pred, average='weighted'),
                "Recall": recall_score(y_test, nn_pred, average='weighted'),
                "ROC-AUC": roc_auc_score(y_test, nn_probs, multi_class='ovr', average='weighted')
        }

        # Display metrics
        print(metrics)
        conf_matrix = confusion_matrix(y_test, nn_pred)
        print("Confusion Matrix:")
        print(conf_matrix)

        return accuracy_score(y_test, nn_pred)

a = neural_network(df, sex="all")
b = neural_network(df, sex="F")
c = neural_network(df, sex="M")

stats['neural_network_all'] = a
stats['neural_network_F'] = b
stats['neural_network_M'] = c

{'Accuracy': 0.6338028169014085, 'F1 Score': np.float64(0.640228339735382), 'Precision': np.float64(0.681992147818686), 'Recall': np.float64(0.6338028169014085), 'ROC-AUC': np.float64(0.7855982046122891)}
Confusion Matrix:
[[15  4  0]
 [ 8 13  2]
 [ 2 10 17]]
{'Accuracy': 0.4722222222222222, 'F1 Score': np.float64(0.4748559670781894), 'Precision': np.float64(0.47850529100529104), 'Recall': np.float64(0.4722222222222222), 'ROC-AUC': np.float64(0.650873281308064)}
Confusion Matrix:
[[7 6 0]
 [3 5 5]
 [2 3 5]]
{'Accuracy': 0.7222222222222222, 'F1 Score': np.float64(0.7139971680826255), 'Precision': np.float64(0.7104938271604938), 'Recall': np.float64(0.7222222222222222), 'ROC-AUC': np.float64(0.8556565656565657)}
Confusion Matrix:
[[13  1  0]
 [ 2  5  4]
 [ 0  3  8]]


In [279]:
def neural_network_regularizada(df, sex="all"):
        """rede neural apresentada está regularizada
        Ridge regularization) is applied through the alpha parameter."""
        if sex=="all":
                filtered_df = df
        elif sex=="F":
                filtered_df = df[df["Sex_F"] > 0.5]
        elif sex=="M":
                filtered_df = df[df["Sex_F"] < 0.5]
        else:
                raise ValueError("Invalid")

        X = filtered_df[['Sex_F', 'Appetite', 'SleepInd', 'StSpirit', 'pEffort',
                'uaI', 'uaI(-1)', 'DeltaWt (%)', 'DeltaHrtwo']]
        y = filtered_df['FatigInd']

        # Split data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


        # Neural Network Model
        nn = MLPClassifier(hidden_layer_sizes=(64, 64), max_iter=250, alpha=0.01) #L2 regularization
        #(also known as Ridge regularization) is applied through the alpha parameter.
        nn.fit(X_train, y_train)
        nn_pred = nn.predict(X_test)
        nn_probs = nn.predict_proba(X_test)  # Probabilities for ROC-AUC

        metrics = {
                "Accuracy": accuracy_score(y_test, nn_pred),
                "F1 Score": f1_score(y_test, nn_pred, average='weighted'),
                "Precision": precision_score(y_test, nn_pred, average='weighted'),
                "Recall": recall_score(y_test, nn_pred, average='weighted'),
                "ROC-AUC": roc_auc_score(y_test, nn_probs, multi_class='ovr', average='weighted')
        }

        # Display metrics
        print(metrics)
        conf_matrix = confusion_matrix(y_test, nn_pred)
        print("Confusion Matrix:")
        print(conf_matrix)

        return accuracy_score(y_test, nn_pred)

a = neural_network_regularizada(df, sex="all")
b = neural_network_regularizada(df, sex="F")
c = neural_network_regularizada(df, sex="M")

stats['neural_network_regularizada_all'] = a
stats['neural_network_regularizada_F'] = b
stats['neural_network_regularizada_M'] = c

{'Accuracy': 0.6056338028169014, 'F1 Score': np.float64(0.6122602608716353), 'Precision': np.float64(0.6232263144081882), 'Recall': np.float64(0.6056338028169014), 'ROC-AUC': np.float64(0.7826897281122633)}
Confusion Matrix:
[[14  7  2]
 [ 4  9  6]
 [ 2  7 20]]
{'Accuracy': 0.6388888888888888, 'F1 Score': np.float64(0.6317901234567901), 'Precision': np.float64(0.6296296296296295), 'Recall': np.float64(0.6388888888888888), 'ROC-AUC': np.float64(0.7051238709453719)}
Confusion Matrix:
[[ 5  0  2]
 [ 1  4  5]
 [ 1  4 14]]
{'Accuracy': 0.6944444444444444, 'F1 Score': np.float64(0.6866422466422466), 'Precision': np.float64(0.689033189033189), 'Recall': np.float64(0.6944444444444444), 'ROC-AUC': np.float64(0.8090520590520591)}
Confusion Matrix:
[[9 1 0]
 [2 7 5]
 [0 3 9]]


/opt/anaconda3/envs/FAA/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/envs/FAA/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/envs/FAA/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(


In [280]:
def SVC_model_free(df, sex="all"):
        """svc model sem regularização (dps ler se é regularizacao e isso o termo correto)"""
        if sex=="all":
                filtered_df = df
        elif sex=="F":
                filtered_df = df[df["Sex_F"] > 0.5]
        elif sex=="M":
                filtered_df = df[df["Sex_F"] < 0.5]
        else:
                raise ValueError("Invalid")

        X = filtered_df[['Sex_F', 'Appetite', 'SleepInd', 'StSpirit', 'pEffort',
                'uaI', 'uaI(-1)', 'DeltaWt (%)', 'DeltaHrtwo']]
        y = filtered_df['FatigInd']

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

        # Create a Support Vector Classifier model
        svc_model = SVC(kernel='linear')  # You can change the kernel to 'rbf', 'poly', etc.

        # Train the model
        svc_model.fit(X_train, y_train)

        # Make predictions on the test set
        y_pred = svc_model.predict(X_test)

        # Evaluate the model
        print("Accuracy:", accuracy_score(y_test, y_pred))
        #print("Classification Report:\n", classification_report(y_test, y_pred))

        return accuracy_score(y_test, y_pred)

a = SVC_model_free(df, sex="all")
b = SVC_model_free(df, sex="F")
c = SVC_model_free(df, sex="M")

stats['SVC_model_free_all'] = a
stats['SVC_model_free_F'] = b
stats['SVC_model_free_M'] = c

Accuracy: 0.6338028169014085
Accuracy: 0.6111111111111112
Accuracy: 0.6111111111111112


In [281]:
def SVC_model_regularizado(df, sex="all"):
        """svc model COM regularização (dps ler se é regularizacao e isso o termo correto)"""
        if sex=="all":
                filtered_df = df
        elif sex=="F":
                filtered_df = df[df["Sex_F"] > 0.5]
        elif sex=="M":
                filtered_df = df[df["Sex_F"] < 0.5]
        else:
                raise ValueError("Invalid")

        X = filtered_df[['Sex_F', 'Appetite', 'SleepInd', 'StSpirit', 'pEffort',
                'uaI', 'uaI(-1)', 'DeltaWt (%)', 'DeltaHrtwo']]
        y = filtered_df['FatigInd']

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

        # Create a Support Vector Classifier model
        svc_model = SVC(kernel='linear')  # You can change the kernel to 'rbf', 'poly', etc.
        svc_model_high_C = SVC(C=1000, kernel='linear')  # High penalty (focuses on reducing training error)
        svc_model_low_C = SVC(C=0.01, kernel='linear')  # Low penalty (allows more errors for larger margin)

        svc_model_high_C.fit(X_train, y_train)
        svc_model_low_C.fit(X_train, y_train)

        # Predictions
        y_pred_high_C = svc_model_high_C.predict(X_test)
        y_pred_low_C = svc_model_low_C.predict(X_test)

        # Evaluate accuracy
        print("Accuracy with high C (10):", accuracy_score(y_test, y_pred_high_C))
        print("Accuracy with low C (0.01):", accuracy_score(y_test, y_pred_low_C))

        return accuracy_score(y_test, y_pred_high_C), accuracy_score(y_test, y_pred_low_C)

a, a1 = SVC_model_regularizado(df, sex="all")
b, b2= SVC_model_regularizado(df, sex="F")
c, c3 = SVC_model_regularizado(df, sex="M")

stats['SVC_model_regularizado_all_high_C'] = a
stats['SVC_model_regularizado_all_low_C'] = a1
stats['SVC_model_regularizado_F_high_C'] = b
stats['SVC_model_regularizado_F_low_C'] = b2
stats['SVC_model_regularizado_M_high_C'] = c
stats['SVC_model_regularizado_M_low_C'] = c3

Accuracy with high C (10): 0.5915492957746479
Accuracy with low C (0.01): 0.49295774647887325
Accuracy with high C (10): 0.5555555555555556
Accuracy with low C (0.01): 0.2777777777777778
Accuracy with high C (10): 0.7222222222222222
Accuracy with low C (0.01): 0.6666666666666666


In [282]:
df = pd.read_excel('results.xlsx')
df = pd.concat([df, pd.DataFrame(stats, index=[0])], ignore_index=True)
df.to_excel('results.xlsx', index=False)